In [1]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()


0

In [2]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/third-experiments/domain_adaptation_project/mlm', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/third-experiments/domain_adaptation_project/modules']


2024-06-04 15:33:53.345518: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-04 15:33:53.427868: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 15:33:54.251170: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from datasets import load_from_disk,concatenate_datasets


adapter_name="mlm_inv_G"
dataset = load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/datasets")



In [5]:
reload(config)

    
filtered_target = dataset['train'].filter(lambda example: example['genre'] == "government").shuffle(seed=42)
#filtered_target = shuffled_filtered_target.train_test_split(test_size=0.1)


filtered_test_target = dataset['validation_matched'].filter(lambda example: example['genre'] == "government")
train_target = filtered_target
test_target = filtered_test_target
train_target[1]
 

{'promptID': 34888,
 'pairID': '34888e',
 'premise': 'The benefits to be gained in air quality were calculated by EPA for the first six years (including the interim program) to be reductions of 125,000 tons of hydrocarbons, 2,388,000 tons of carbon monoxide and 450,000 tons of nitrogen oxide.',
 'premise_binary_parse': '( ( The ( benefits ( to ( be ( gained ( in ( air quality ) ) ) ) ) ) ) ( ( were ( ( ( ( calculated ( by EPA ) ) ( for ( the ( first ( six years ) ) ) ) ) ( -LRB- ( ( including ( the ( interim program ) ) ) -RRB- ) ) ) ( to ( be ( reductions ( of ( ( ( ( ( ( 125,000 tons ) ( of hydrocarbons ) ) , ) ( ( 2,388,000 tons ) ( of ( carbon monoxide ) ) ) ) and ) ( ( 450,000 tons ) ( of ( nitrogen oxide ) ) ) ) ) ) ) ) ) ) . ) )',
 'premise_parse': '(ROOT (S (NP (DT The) (NNS benefits) (S (VP (TO to) (VP (VB be) (VP (VBN gained) (PP (IN in) (NP (NN air) (NN quality)))))))) (VP (VBD were) (VP (VBN calculated) (PP (IN by) (NP (NNP EPA))) (PP (IN for) (NP (DT the) (JJ first) (CD si

In [6]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoModelForMaskedLM.from_pretrained(
    config.Config.MODEL_NAME,
)
init(model)
reload(fn)
fn.print_trainable_parameters(model)

/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertFor

trainable params: 109514298 || all params: 109514298 || trainable%: 100.0


In [7]:
from adapters import ConfigUnion, LoRAConfig, PrefixTuningConfig, SeqBnConfig,SeqBnInvConfig,AdapterConfig,LoRAConfig


adapter_config = SeqBnInvConfig(reduction_factor=2)

model.add_adapter(adapter_name, config=adapter_config)

model.train_adapter([adapter_name])
model.active_adapters = adapter_name
print(model.adapter_summary())
fn.print_trainable_parameters(model)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
mlm_inv_G                bottleneck        7,387,776       6.785       1       1
--------------------------------------------------------------------------------
Full model                               108,891,648     100.000               0
trainable params: 7979904 || all params: 116902074 || trainable%: 6.82614407679371


In [8]:
from transformers import AutoTokenizer

reload(processed)
tokenizer = AutoTokenizer.from_pretrained(config.Config.TOKENIZER_NAME)

tokenized_dataset= processed.tokenize_dataset(train_target,tokenizer)
tokenized_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
    num_rows: 77350
})

In [9]:
chunk_size = 128
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset[444:470]

results = fn.group_texts(tokenized_samples, chunk_size)
for chunk in results["labels"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'


In [10]:
# we group texts and chunk them
lm_datasets = tokenized_dataset.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
lm_datasets

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 25770
})

In [11]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
samples = [lm_datasets[i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")
dd=data_collator(samples)
for chunk in dd["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] some investments in stewardrada land, for example national parks, will be reported by both 1 ) the number of acres used [MASK] [MASK] park or an historic site in the stewardship [MASK] sarcastically, and 2 ) by the number of physical units identified as national parks in the [MASK] assets category. [SEP] [MASK] number [MASK] acres used and [MASK] [MASK] [MASK] physical units [MASK] on [MASK]ship land are both reported. [SEP] [CLS] the benefits to [MASK] gained in [MASK] quality were calculated wholly epa for the first six [MASK] ( including the interim program ) to be reductions of [MASK] [MASK] 000 tons of hydro doubts [MASK], 2, 388, 000 tons of carbon mono [MASK]'

'>>> [MASK] 450, 000 tons of nitrogen oxide. [SEP] epa estimate that, in the first six drunken, there will be 125 [MASK] motive fewer tons of hydrocarbons. [SEP] [CLS] 6 trillion. [SEP] organising trillion [SEP] [CLS] professional training organizations [MASK] offer on - campus training [MASK] very [SEP] there

In [12]:

downsampled_dataset = lm_datasets.train_test_split(
    test_size=0.1, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 23193
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 2577
    })
})

In [13]:

reload(fn)
trainer = fn.train_mlm_model(model,adapter_name,data_collator,tokenizer, downsampled_dataset['train'],downsampled_dataset['test'])

/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [14]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/81 [00:00<?, ?it/s]

>>> Perplexity: 16.32


In [15]:
trainer.train()

  0%|          | 0/14500 [00:00<?, ?it/s]

{'loss': 2.3227, 'grad_norm': 1.9832983016967773, 'learning_rate': 9.84e-05, 'epoch': 1.0}


  0%|          | 0/81 [00:00<?, ?it/s]

{'eval_loss': 2.014267921447754, 'eval_runtime': 4.1428, 'eval_samples_per_second': 622.04, 'eval_steps_per_second': 19.552, 'epoch': 1.0}
{'loss': 2.0522, 'grad_norm': 1.7894372940063477, 'learning_rate': 9.322857142857144e-05, 'epoch': 2.0}


  0%|          | 0/81 [00:00<?, ?it/s]

...

  0%|          | 0/81 [00:00<?, ?it/s]

{'eval_loss': 1.6396472454071045, 'eval_runtime': 4.2582, 'eval_samples_per_second': 605.19, 'eval_steps_per_second': 19.022, 'epoch': 20.0}


Overwriting existing adapter 'mlm_inv_G'.


{'train_runtime': 1701.892, 'train_samples_per_second': 272.555, 'train_steps_per_second': 8.52, 'train_loss': 1.7976219748135271, 'epoch': 20.0}


TrainOutput(global_step=14500, training_loss=1.7976219748135271, metrics={'train_runtime': 1701.892, 'train_samples_per_second': 272.555, 'train_steps_per_second': 8.52, 'train_loss': 1.7976219748135271, 'epoch': 20.0})

In [16]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/81 [00:00<?, ?it/s]

>>> Perplexity: 4.99


In [17]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)


: 

#### This is a report of mlm for goverment domain using whole genre as target and then splitt after tokenization with seed 42. I used computer metrix for preplexity and early stopping 
The results before:
>>> Perplexity: 16.32

The results after:
>>> Perplexity: 4.99